In [2]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import config
import os.path
from os import path

from create_db import create_connection, create_table, clean_column_names
from populate_db import extract_data, insert_records
import query_db as qd

import importlib

In [3]:
importlib.reload(qd)

<module 'query_db' from '/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/query_db.py'>

In [4]:
import datetime

In [5]:
import re

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
start = datetime.datetime.now()
query_court_commitment = '''
                        SELECT A.OFFENDER_NC_DOC_ID_NUMBER as ID, 
                            A.COMMITMENT_PREFIX, 
                            A.EARLIEST_SENTENCE_EFFECTIVE_DT, 
                            A.MOST_SERIOUS_OFFENSE_CODE                              
                        FROM OFNT3BB1 A
                        WHERE NEW_PERIOD_OF_INCARCERATION_FL = "Y";
                        '''

conn = create_connection(config.database_name)
court_small = qd.query_db_notebook(conn,query_court_commitment)


query_sentence_comp = '''
                            SELECT INMATE_DOC_NUMBER as ID, 
                                INMATE_COMMITMENT_PREFIX as COMMITMENT_PREFIX, 
                                INMATE_COMPUTATION_STATUS_FLAG, 
                                max(ACTUAL_SENTENCE_END_DATE) as END_DATE
                            FROM INMT4BB1
                            GROUP BY INMATE_DOC_NUMBER, INMATE_COMMITMENT_PREFIX;
                        '''

sentence_compute_small = qd.query_db_notebook(conn,query_sentence_comp)

conn.close

data = court_small.merge(sentence_compute_small, on=['ID','COMMITMENT_PREFIX'], how='outer')

stop = datetime.datetime.now()
print("Time Elapsed:", stop - start) 


Time Elapsed: 0:00:58.509630


In [6]:
data.to_csv('datasetA_court_sentcomp.csv', index=False)

In [67]:
data.shape

(903181, 5)

In [95]:
data.groupby('INMATE_COMPUTATION_STATUS_FLAG').count()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,END_DATE
INMATE_COMPUTATION_STATUS_FLAG,,,,,
ACTIVE,27039,27039,27009,27009,27039
EAR.TERM,2970,2970,2897,2896,2970
EXPIRED,853716,853716,853022,530070,853716
FUTURE,6184,6184,6164,6164,6184
PAROLED,198,198,197,197,198
POST REL,11797,11797,11777,11777,11797
UNKNOWN,15,15,15,15,15


In [83]:
# Missing most serious offense code  - 324,308 - about a third of the data :(
data['MOST_SERIOUS_OFFENSE_CODE'].isnull().sum()

324308

In [96]:
# data in court commitment but not sentence computation - ~ 1,200
# I spot checked some of these and they are  ACTIVE sentences, whereas others just seem to be weird data quirks..
data[data['END_DATE'].isnull()].head()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE
114,0000156,02,1995-03-15,RIOT & CIVIL DISORDERS,NaN,NaN
287,0000327,BA,1991-05-30,None,NaN,NaN
2867,0002686,BA,1993-12-06,MISD B&E,NaN,NaN
3418,0003172,BA,1989-03-06,None,NaN,NaN
4167,0003770,BA,1995-03-25,LARCENY OF MOTOR VEHICLE,NaN,NaN


In [97]:
data[data['EARLIEST_SENTENCE_EFFECTIVE_DT'].isnull()].head()
# 838 observations that appear in sentence computation but not in court commitment
# i looked up a few of them and it seemed like their full record was unavailable? might be some very recent cases with a
# bit of a time lag. since there are only ~1000 in a dataset of 1 million, i'm not too worried about dropping these

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE
902343,0001133,BA,NaN,NaN,EXPIRED,1996-06-23
902344,0001166,AI,NaN,NaN,EXPIRED,1990-07-06
902345,0001407,BE,NaN,NaN,EXPIRED,2011-07-29
902346,0001640,BA,NaN,NaN,EXPIRED,1998-03-25
902347,0005195,BF,NaN,NaN,EXPIRED,2006-04-06


In [46]:
start = datetime.datetime.now()

query_sentence_component = '''
                            SELECT OFFENDER_NC_DOC_ID_NUMBER as ID, 
                                        COMMITMENT_PREFIX, 
                                        SENTENCE_COMPONENT_NUMBER,
                                        PRIMARY_OFFENSE_CODE,
                                        PRIMARY_FELONYMISDEMEANOR_CD,
                                        SENTENCING_PENALTY_CLASS_CODE,
                                        PRIOR_RECORD_LEVEL_CODE,
                                        MINIMUM_SENTENCE_LENGTH,
                                        MAXIMUM_SENTENCE_LENGTH,
                                        SENTENCE_TYPE_CODE
                            FROM OFNT3CE1
                            WHERE SENTENCE_TYPE_CODE LIKE '%PRISONS%';
                            '''

# query_sentence_component = '''
#                             SELECT OFFENDER_NC_DOC_ID_NUMBER as ID, 
#                                         COMMITMENT_PREFIX, 
#                                         SENTENCE_COMPONENT_NUMBER,
#                                         PRIMARY_OFFENSE_CODE,
#                                         PRIMARY_FELONYMISDEMEANOR_CD,
#                                         SENTENCING_PENALTY_CLASS_CODE,
#                                         PRIOR_RECORD_LEVEL_CODE,
#                                         MINIMUM_SENTENCE_LENGTH,
#                                         MAXIMUM_SENTENCE_LENGTH,
#                                         SENTENCE_TYPE_CODE
#                             FROM OFNT3CE1;
#                             '''

#                            WHERE SENTENCE_TYPE_CODE = 'DEPT OF CORR DIV OF PRISONS';

# query_sentence_component = '''
#                             SELECT SENTENCE_TYPE_CODE,
#                                         COUNT(SENTENCE_TYPE_CODE)
#                             FROM OFNT3CE1
#                             GROUP BY SENTENCE_TYPE_CODE;
#                             '''
# query_sentence_component = '''
#                             SELECT COUNT(*)
#                             FROM OFNT3CE1;
#                             '''


conn = create_connection(config.database_name)
sent_comp_small = qd.query_db_notebook(conn,query_sentence_component)

stop = datetime.datetime.now()
print("Time Elapsed:", stop - start) 


Time Elapsed: 0:08:02.165564


In [63]:
# Sanity check
 # I ran the query with "WHERE SENTENCE_TYPE_CODE LIKE '%PRISONS%'" to capture all sentences that resulted in
    # incarceration (not probation). this results in 1,728,836 rows.  this query runs pretty quickly. however, i wanted to sanity check that
    # this was capturing all commitment prefixes which are letters (e.g. AB, BA, etc) and not numbers. This 
    # should be true if our understanding is correct that letters correspond to incarceration and numbers
    # correspond to probation
    # running the full query takes about 8 minutes, then i restrict to commitment prefixes that are letters
    # the result is also 1,728,836, confirming that it is ok to use the first query since commitment prefixes
    # that are letters correspond to sentence type codes that result in incarceration


sent_comp_small_prefix = sent_comp_small[sent_comp_small['COMMITMENT_PREFIX'].str.contains('[a-zA-Z]')]

In [62]:
sent_comp_small_prefix.shape

(1728836, 10)

In [42]:
sent_comp_small.to_csv('datasetB_sentcomponent_only_incarcerated.csv', index=False)

In [38]:
# 4,344,946 total rows in sentence component
# 1,728,836 Dept of corr div of prisons

# query_court_commitment = '''
#                         SELECT NEW_PERIOD_OF_INCARCERATION_FL, COUNT(NEW_PERIOD_OF_INCARCERATION_FL)
#                         FROM OFNT3BB1
#                         GROUP BY NEW_PERIOD_OF_INCARCERATION_FL;
#                         '''

In [90]:
# query_sentence_computation = '''
#                             SELECT INMATE_DOC_NUMBER as ID, 
#                                    INMATE_COMMITMENT_PREFIX as COMMITMENT_PREFIX, 
#                                    INMATE_SENTENCE_COMPONENT, 
#                                    ACTUAL_SENTENCE_END_DATE
#                             FROM INMT4BB1
#                             WHERE INMATE_DOC_NUMBER = '0002686';
#                             '''

# query_sentence_computation = '''
#                             SELECT *
#                             FROM INMT4BB1
#                             WHERE INMATE_COMPUTATION_STATUS_FLAG = "ACTIVE"
#                             LIMIT 1000;
#                             '''


# query_sentence_computation = '''
#                             SELECT INMATE_COMPUTATION_STATUS_FLAG, 
#                                    COUNT(INMATE_COMPUTATION_STATUS_FLAG) 
#                             FROM INMT4BB1
#                             GROUP BY INMATE_COMPUTATION_STATUS_FLAG;
#                             '''



In [91]:
# conn = create_connection(config.database_name)
# sentence_computation2 = qd.query_db_notebook(conn,query_sentence_computation)
# conn.close

<function Connection.close>

In [28]:
# # Working query, but takes a LONG time to run. much quick to query data separately then join
# query_court_commitment = '''
#                         SELECT A.OFFENDER_NC_DOC_ID_NUMBER as ID, 
#                                 A.COMMITMENT_PREFIX, 
#                                 A.EARLIEST_SENTENCE_EFFECTIVE_DT, 
#                                 A.MOST_SERIOUS_OFFENSE_CODE, 
#                                 (SELECT max(ACTUAL_SENTENCE_END_DATE)
#                                     FROM INMT4BB1 B                                    
#                                     WHERE A.OFFENDER_NC_DOC_ID_NUMBER = B.INMATE_DOC_NUMBER
#                                     AND A.COMMITMENT_PREFIX = B.INMATE_COMMITMENT_PREFIX
#                                     GROUP BY INMATE_DOC_NUMBER, INMATE_COMMITMENT_PREFIX) as END_DATE                                
#                         FROM OFNT3BB1 A
#                         WHERE NEW_PERIOD_OF_INCARCERATION_FL = "Y"
#                         LIMIT 1000;
#                         '''

## Charmaine's WIP code

To-do:
1. Check the number of unique primary offense codes for every combination of ID and COMMITMENT_PREFIX
2. Check what is going on with the 9999999 values in MINIMUM_SENTENCE_LENGTH
3. Spot check IDs in court commitment data to compare most serious offense

In [1]:
# Import CSVs
import pandas as pd

dataset_A = pd.read_csv('../data/datasetA_court_sentcomp.csv')
dataset_B = pd.read_csv('../data/datasetB_sentcomponent_only_incarcerated.csv')

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
dataset_A.head()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE
0,4,AA,1983-07-12,NaN,EXPIRED,1984-07-11
1,6,AA,1973-01-30,NaN,EXPIRED,1973-03-28
2,6,AB,1973-04-11,NaN,EXPIRED,1975-08-18
3,8,AA,1990-04-09,NaN,EXPIRED,1990-05-17
4,8,AB,1993-08-30,NaN,EXPIRED,1994-01-26


In [3]:
dataset_A.columns

Index(['ID', 'COMMITMENT_PREFIX', 'EARLIEST_SENTENCE_EFFECTIVE_DT',
       'MOST_SERIOUS_OFFENSE_CODE', 'INMATE_COMPUTATION_STATUS_FLAG',
       'END_DATE'],
      dtype='object')

In [4]:
dataset_B.head()

,ID,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,PRIMARY_OFFENSE_CODE,PRIMARY_FELONYMISDEMEANOR_CD,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,SENTENCE_TYPE_CODE
0,4,AA,1,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
1,4,AA,2,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
2,6,AA,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,60,90,DEPT OF CORR DIV OF PRISONS
3,6,AB,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
4,6,AB,2,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS


In [5]:
dataset_B.columns

Index(['ID', 'COMMITMENT_PREFIX', 'SENTENCE_COMPONENT_NUMBER',
       'PRIMARY_OFFENSE_CODE', 'PRIMARY_FELONYMISDEMEANOR_CD',
       'SENTENCING_PENALTY_CLASS_CODE', 'PRIOR_RECORD_LEVEL_CODE',
       'MINIMUM_SENTENCE_LENGTH', 'MAXIMUM_SENTENCE_LENGTH',
       'SENTENCE_TYPE_CODE'],
      dtype='object')

In [6]:
dataset_A.shape

(903181, 6)

In [7]:
dataset_B.shape

(1728836, 10)

In [8]:
# Group B by ID and COMMITMENT_PREFIX, find maximum of MINIMUM_SENTENCE_LENGTH
check_minsentence = pd.DataFrame(dataset_B.groupby(['ID', 'COMMITMENT_PREFIX','PRIMARY_FELONYMISDEMEANOR_CD', 'PRIMARY_OFFENSE_CODE'])['MINIMUM_SENTENCE_LENGTH'].max().reset_index(name='max'))
check_minsentence.head(10)

,ID,COMMITMENT_PREFIX,PRIMARY_FELONYMISDEMEANOR_CD,PRIMARY_OFFENSE_CODE,max
0,4,AA,FELON,SELL SCHEDULE II,0
1,6,AA,MISD.,WORTHLESS CHECK,60
2,6,AB,MISD.,WORTHLESS CHECK,30
3,8,AA,MISD.,DWI DRIVING WHILE IMPAIRED,10000
4,8,AB,FELON,HABITUAL IMPAIRED DRIVING,0
5,8,BA,FELON,HABITUAL IMPAIRED DRIVING,0
6,10,AA,FELON,SAFECRACKING/SAFE ROBBERY,0
7,10,AB,FELON,B & E & L,100000
8,10,AB,FELON,LARCENY OF MOTOR VEHICLE,50000
9,10,AB,FELON,SAFECRACKING/SAFE ROBBERY,200000


In [22]:
min_sentence = pd.DataFrame(dataset_B.groupby(['ID', 'COMMITMENT_PREFIX'])['MINIMUM_SENTENCE_LENGTH'].max().reset_index(name='max_min'))
min_sentence.head(10)

,ID,COMMITMENT_PREFIX,max_min
0,4,AA,0
1,6,AA,60
2,6,AB,30
3,8,AA,10000
4,8,AB,0
5,8,BA,0
6,10,AA,0
7,10,AB,200000
8,14,AA,1800
9,14,AB,600


In [10]:
# Filter dataset_B to only these rows
filter_tuples = [tuple(x) for x in min_sentence.to_numpy()]

filtered_B = dataset_B[dataset_B[['ID', 'COMMITMENT_PREFIX', 'MINIMUM_SENTENCE_LENGTH']].apply(tuple, axis=1).isin(filter_tuples)]
filtered_B.head(10)

,ID,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,PRIMARY_OFFENSE_CODE,PRIMARY_FELONYMISDEMEANOR_CD,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,SENTENCE_TYPE_CODE
0,4,AA,1,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
1,4,AA,2,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
2,6,AA,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,60,90,DEPT OF CORR DIV OF PRISONS
3,6,AB,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
4,6,AB,2,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
5,6,AB,3,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
6,6,AB,4,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
7,6,AB,5,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
8,6,AB,6,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
9,6,AB,7,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS


Based on the sample of observations above, there are quite a few ties, but it might be that they are all of the same PRIMARY_OFFENSE_CODE (e.g., rows 3-9). I will check how many unique primary offense codes there are for every ID and COMMITMENT_PREFIX combination.

In [11]:
unique_offenses = pd.DataFrame(filtered_B.groupby(['ID', 'COMMITMENT_PREFIX'])['PRIMARY_OFFENSE_CODE'].nunique().reset_index(name='count'))
unique_offenses['count'].describe()

count    891122.000000
mean          1.199647
std           0.547120
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          12.000000
Name: count, dtype: float64

In [19]:
unique_offenses.head(1)

,ID,COMMITMENT_PREFIX,count
0,4,AA,1


It looks like the vast majority of ID and COMMITMENT_PREFIX combinations have a single PRIMARY_OFFENSE_CODE, but there is at least one combination that had 12 unique PRIMARY_OFFENSE_CODES.

In [12]:
more_than_one_offense = unique_offenses[unique_offenses['count'] > 1].sort_values('count')
more_than_one_offense.head(10)

,ID,COMMITMENT_PREFIX,count
11,19,AA,2
482426,433706,AB,2
482420,433699,BB,2
482410,433692,BC,2
482409,433692,BB,2
482400,433688,AB,2
482391,433686,AA,2
482360,433668,AD,2
482353,433660,BA,2
482338,433641,AC,2


In [13]:
more_than_one_offense.shape

(134415, 3)

There are 134,415 observations (ID and COMMITMENT_PREFIX combinations) that have more than one unique PRIMARY_OFFENSE_CODE. Check what PRIMARY_OFFENSE_CODES are associated with these observations (in dataset B), and check (if available) what their MOST_SERIOUS_OFFENSE was (in dataset A).

In [14]:
offenses_filter = [tuple(x) for x in more_than_one_offense[['ID', 'COMMITMENT_PREFIX']].to_numpy()]

filtered_B_mt1 = filtered_B[filtered_B[['ID', 'COMMITMENT_PREFIX']].apply(tuple, axis=1).isin(offenses_filter)]
filtered_B_mt1.head()

,ID,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,PRIMARY_OFFENSE_CODE,PRIMARY_FELONYMISDEMEANOR_CD,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,SENTENCE_TYPE_CODE
41,19,AA,1,MISD B&E,MISD.,MISD.(PRE-STRUCTURE),NaN,2400,2400,DEPT OF CORR DIV OF PRISONS
42,19,AA,2,LARCENY (OVER $200),MISD.,MISD.(PRE-STRUCTURE),NaN,2400,2400,DEPT OF CORR DIV OF PRISONS
45,20,BA,1,FELONY B&E,FELON,CLASS H,NaN,0,100000,DEPT OF CORR DIV OF PRISONS
46,20,BA,2,B & E & L,FELON,CLASS H,NaN,0,100000,DEPT OF CORR DIV OF PRISONS
47,20,BA,3,FELONY B&E,FELON,CLASS H,NaN,0,60000,DEPT OF CORR DIV OF PRISONS


We have some odd observations where the primary offense is a felony, but MINIMUM_SENTENCE_LENGTH is 0 and MAXIMUM_SENTENCE_LENGTH > 0 (e.g., index 45-47)

In [15]:
filtered_B_mt1 = filtered_B_mt1[['ID', 'COMMITMENT_PREFIX', 'PRIMARY_OFFENSE_CODE', 'MINIMUM_SENTENCE_LENGTH', 'MAXIMUM_SENTENCE_LENGTH']]
filtered_B_mt1

,ID,COMMITMENT_PREFIX,PRIMARY_OFFENSE_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH
41,19,AA,MISD B&E,2400,2400
42,19,AA,LARCENY (OVER $200),2400,2400
45,20,BA,FELONY B&E,0,100000
46,20,BA,B & E & L,0,100000
47,20,BA,FELONY B&E,0,60000
74,33,AD,ROBBERY W/DANGEROUS WEAPON,0,180000
75,33,AD,ESCAPE PRISON,0,30000
83,35,BA,FELONY B&E,0,80000
84,35,BA,B & E & L,0,30000
103,49,AC,INDECENT LIBERTY W/CHILD,0,60000


In [21]:
find_max_max = pd.DataFrame(filtered_B_mt1.groupby(['ID', 'COMMITMENT_PREFIX'])['MAXIMUM_SENTENCE_LENGTH'].max().reset_index(name='max_max'))
find_max_max.sample(5)

,ID,COMMITMENT_PREFIX,max_max
59641,300717,AC,20000
15793,79118,AA,30000
68125,344547,BB,10500
17307,87117,AA,30000
27050,136181,AC,20000


In [26]:
by_max_tuples = [tuple(x) for x in find_max_max.to_numpy()]
filtered_max = filtered_B_mt1[filtered_B_mt1[['ID', 'COMMITMENT_PREFIX', 'MAXIMUM_SENTENCE_LENGTH']].apply(tuple, axis=1).isin(by_max_tuples)]
filtered_max.head()

,ID,COMMITMENT_PREFIX,PRIMARY_OFFENSE_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH
41,19,AA,MISD B&E,2400,2400
42,19,AA,LARCENY (OVER $200),2400,2400
45,20,BA,FELONY B&E,0,100000
46,20,BA,B & E & L,0,100000
74,33,AD,ROBBERY W/DANGEROUS WEAPON,0,180000


In [30]:
#filtered_max
choose_random = pd.DataFrame(filtered_max.groupby(['ID', 'COMMITMENT_PREFIX']).size().reset_index(name='count'))
choose_random = choose_random[choose_random['count'] > 1][['ID', 'COMMITMENT_PREFIX']]

In [32]:
choose_random.shape

(72137, 2)

There are 72,137 ID and COMMITMENT_PREFIX combinations we have to choose randomly from. 

In [33]:
choose_random.head()

,ID,COMMITMENT_PREFIX
0,19,AA
1,20,BA
7,74,AA
8,74,AB
9,80,BA


In [16]:
# What do we do with MINIMUM_SENTENCE_LENGTH = 9999999
dataset_B['MINIMUM_SENTENCE_LENGTH'].describe()

count    1.728836e+06
mean     2.569733e+04
std      4.212079e+05
min      0.000000e+00
25%      0.000000e+00
50%      4.000000e+02
75%      1.200000e+03
max      9.999999e+06
Name: MINIMUM_SENTENCE_LENGTH, dtype: float64

#### What are the offenses associated with 9999999 minimum sentence lengths?

Based on the grouped dataframe below, most of the offenses are high-risk felonies, so likely equivalent to a life sentence.

In [17]:
pd.DataFrame(dataset_B[dataset_B['MINIMUM_SENTENCE_LENGTH'] == 9999999].groupby('PRIMARY_OFFENSE_CODE').size().reset_index(name='count'))

,PRIMARY_OFFENSE_CODE,count
0,BURGLARY 1ST DEGREE,1
1,COMMON LAW ROBBERY,1
2,MURDER FIRST DEGREE,448
3,RAPE FIRST DEGREE,9
4,SEXUAL OFFENSE 1ST DEGREE,1


I also checked the MAXIMUM_SENTENCE_LENGTH column to see what that looked like: either 9999999 or 0, so that squares with the hypothesis that these are life sentences. 

In [18]:
dataset_B[dataset_B['MINIMUM_SENTENCE_LENGTH'] == 9999999]['MAXIMUM_SENTENCE_LENGTH'].unique()

array([9999999, 0, '9999999'], dtype=object)

#### Main Takeaways:
* Choosing observations (ID and COMMITMENT_PREFIX combination) based on sentence length:
    * Choose the observation that has the maximum MINIMUM_SENTENCE_LENGTH
    * If there is a tie on MINIMUM_SENTENCE_LENGTH, select observation with the maximum MAXIMUM_SENTENCE_LENGTH 
    * If there is a tie on MAXIMUM_SENTENCE_LENGTH, then choose randomly
* Do not change 9999999 values to NA, since these likely correspond to life-sentences. Check against court commitment data.

In [63]:
# Finalize dataset_B
seed = 10000
# unique_offenses gives us the where ID+COMMITMENT_PREFIX have a unique max on MINIMUM_SENTENCE_LENGTH
# filtered_B_mt1 give us the records where ID+COMMITMENT_PREFIX does not have a unique max on MINIMUM_SENTENCE_LENGTH

# (MINIMUM_SENTENCE_LENGTH = max_min and unique_min = False and MAXIMUM_SENTENCE_LENGTH = max_max and unique_max = True) or
# (MINIMUM_SENTENCE_LENGTH = max_min and unique_min = False and 
#  MAXIMUM_SENTENCE_LENGTH = max_max and unique_max = False).sample(1, random_state=seed)


,ID,COMMITMENT_PREFIX,SENTENCE_COMPONENT_NUMBER,PRIMARY_OFFENSE_CODE,PRIMARY_FELONYMISDEMEANOR_CD,SENTENCING_PENALTY_CLASS_CODE,PRIOR_RECORD_LEVEL_CODE,MINIMUM_SENTENCE_LENGTH,MAXIMUM_SENTENCE_LENGTH,SENTENCE_TYPE_CODE
0,4,AA,1,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
1,4,AA,2,SELL SCHEDULE II,FELON,CLASS H,NaN,0,30000,DEPT OF CORR DIV OF PRISONS
2,6,AA,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,60,90,DEPT OF CORR DIV OF PRISONS
3,6,AB,1,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS
4,6,AB,2,WORTHLESS CHECK,MISD.,MISD.(PRE-STRUCTURE),NaN,30,0,DEPT OF CORR DIV OF PRISONS


In [ ]:
# Puling it all together - get most serious offense code for each id and commitment prefix
merged = dataset_A.merge(dataset_B, how='inner', on=['ID', 'COMMITMENT_PREFIX'])